# Cloud Run Test

In [ ]:
# Simple App

In [ ]:
!mkdir cloud-run-test

In [ ]:
cd cloud-run-test

In [ ]:
%%writefile app.py
import streamlit as st

st.write("It's working. v2")

In [ ]:
%%writefile Dockerfile
FROM python
WORKDIR /app
RUN pip install streamlit -q
COPY . /app
EXPOSE 9000
CMD ["streamlit", "run", "app.py", "--server.address", "0.0.0.0", "--server.port", "9000"]

In [ ]:
!docker build -t simple-app . -q

In [ ]:
# List images
!docker images

In [ ]:
# Run the app in detached mode
!docker run -d -p 9000:9000 -v $(pwd):/app --name simple-app simple-app

In [ ]:
!docker ps -a

In [ ]:
# Remove all docker ps
# !docker rm -f $(docker ps -aq)

In [ ]:
!docker rm -f simple-app

### Deployment

#### Deploy using Cloud Build, Artifact Registry, and Cloud Run

In [ ]:
%%sh
# Create a Docker repository in Artifact Registry
gcloud artifacts repositories create simple-app \
    --repository-format=docker \
    --location=us-west2 \
    --description="Docker repository"

In [ ]:
%%sh
# Verify
gcloud artifacts repositories list

#### Build an image using Dockerfile

In [ ]:
# Check if the path is correct
!pwd

In [ ]:
cd cloud-run-test

In [ ]:
!gcloud builds submit \
    --region=us-west2 \
    --tag us-west2-docker.pkg.dev/$(gcloud config get-value project)/simple-app/simple-app:tag1

#### Pull the image from Artifact Registry

In [ ]:
# !gcloud auth configure-docker us-west2-docker.pkg.dev

In [ ]:
!docker pull us-west2-docker.pkg.dev/$(gcloud config get-value project)/app-v2/app:tag1

In [ ]:
!docker images

In [ ]:
!docker run -d -p 8501:8501 -v $(pwd):/app us-west2-docker.pkg.dev/$(gcloud config get-value project)/app-v2/app:tag1

In [ ]:
!docker ps

In [ ]:
# !docker rm -f <...>